<a href="https://colab.research.google.com/github/Prisci723/InteligenciaArtificial1/blob/main/Laboratorio%205%20Redes%20neuronales/Lab5_2_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Redes Neuronales con Pytorch

## Introduction

El dataset será el mismo que ya ha sido procesado en el anterior

https://colab.research.google.com/drive/1yWS2sEmPpC0NWDP72EC8g0Mo6638Ck6w?usp=sharing

Como el dataset ya está listo es que se procede a aplicar el modelo, esta vez aplicando redes neuronales con pytorch

In [2]:
# SE IMPORTAN LAS LIBRERÍAS NECESARIAS
import os
# Cálculo científico y vectorial para python
import numpy as np
# Libreria para graficos
from matplotlib import pyplot as plt


import torch
from torch import optim
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import torchvision.datasets as datasets
from tqdm import tqdm

# le dice a matplotlib que incruste gráficos en el cuaderno
%matplotlib inline

Se añade el acceso a google drive

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Se crea una red neuronal simple utilizando subclases y herencia, se definen las capas que tendrá la red y la cantidad de perceptrones en la capa oculta, que en este caso son 50

In [4]:

class RedNeuronalMLS(nn.Module):
    def __init__(self, input_size, num_classes):
        super(RedNeuronalMLS, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)

    def forward(self, x):
        x = self.fc1(x)
        x = F.sigmoid(x)
        x = self.fc2(x)
        return x

Definimos el recurso que se utilizará para procesar los datos, también los hiperparámetros con los datos correpondientes al dataset y los que mejor permitan optimizar el modelo

In [5]:
# Se utiliza el recurso que este disponible GPU o CPU por defecto
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Hiperparámetros
input_size = 54
num_classes = 7
learning_rate = 0.01
batch_size = 10000
num_epochs = 100

cpu


Cargamos los datos de Google Drive

In [6]:

data = np.loadtxt('/content/gdrive/MyDrive/Machine learning/Datasets/covtype.csv', delimiter=',')
Y = data[:, -1]  # Última columna
X = data[:, :-1]  # Resto de las columnas


#Los valores de y cambian para que comienzen en 0
Y[Y == 1] = 0
Y[Y == 2] = 1
Y[Y == 3] = 2
Y[Y == 4] = 3
Y[Y == 5] = 4
Y[Y == 6] = 5
Y[Y == 7] = 6

#Obtenemos el 80% de los datos y separamos el entrenamiento de la prueba
test = X.shape[0]*0.8
test = round(test)
X_train =  X[:test]
X_test = X[test:]
print(len(X_train))
print(len(X_test))
y_train = Y[:test]
y_test =Y[test:]

464810
116202


Los datos de X son normalizados para facilitar su procesamiento

In [7]:
def  featureNormalize(X):
    X_norm = X.copy()
    mu = np.zeros(X.shape[1])
    sigma = np.zeros(X.shape[1])

    mu = np.mean(X, axis = 0)
    sigma = np.std(X, axis = 0)
    X_norm = (X - mu) / sigma

    return X_norm, mu, sigma

In [8]:
X_train, mu, sigma = featureNormalize(X_train)

In [9]:
def  testNormalize(X, mu, sigma):
    X_norm = X.copy()
    X_norm = (X - mu) / sigma
    m, n = X_norm.shape
    return X_norm

In [10]:
X_test = testNormalize(X_test, mu, sigma)

**Convertimos el dataset en tensores**

Para poder aplicar el modelo en pytorch es que necesitamos convertir el dataset en tensores de PyTorch

In [11]:

# Convertir los arrays NumPy en tensores de PyTorch
X_tensor = torch.tensor(X_train, dtype=torch.float32)
y_tensor = torch.tensor(y_train, dtype=torch.long)
X_tensor_t = torch.tensor(X_test, dtype=torch.float32)
y_tensor_t = torch.tensor(y_test, dtype=torch.long)

In [12]:
dataset = TensorDataset(X_tensor, y_tensor)
dataset_test = TensorDataset(X_tensor_t, y_tensor_t)

Se inicializa la red

In [13]:
model = RedNeuronalMLS(input_size=input_size, num_classes=num_classes).to(device)
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset_test, batch_size=batch_size, shuffle=True)

In [14]:
model

RedNeuronalMLS(
  (fc1): Linear(in_features=54, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=7, bias=True)
)

Se define la función de costo y el opmtimizador en este caso, se utiliza como funcion de costo el Cross Entropy y la función de optimización Adam

In [15]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

La red entrenada, según como se definió el modelo anteriormente

In [16]:

for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
    # el dataset es cargado en forma de tensores

        data = data.to(device=device)
        targets = targets.to(device=device)

        # Se obtiene el tamaño de los datos
        data = data.reshape(data.shape[0], -1)
        print(data.shape)
        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()

        # aplicación de la optimización
        optimizer.step()

torch.Size([10000, 54])
torch.Size([10000, 54])
torch.Size([10000, 54])
torch.Size([10000, 54])
torch.Size([10000, 54])
torch.Size([10000, 54])
torch.Size([10000, 54])
torch.Size([10000, 54])
torch.Size([10000, 54])
torch.Size([10000, 54])
torch.Size([10000, 54])
torch.Size([10000, 54])
torch.Size([10000, 54])
torch.Size([10000, 54])
torch.Size([10000, 54])
torch.Size([10000, 54])
torch.Size([10000, 54])
torch.Size([10000, 54])
torch.Size([10000, 54])
torch.Size([10000, 54])
torch.Size([10000, 54])
torch.Size([10000, 54])
torch.Size([10000, 54])
torch.Size([10000, 54])
torch.Size([10000, 54])
torch.Size([10000, 54])
torch.Size([10000, 54])
torch.Size([10000, 54])
torch.Size([10000, 54])
torch.Size([10000, 54])
torch.Size([10000, 54])
torch.Size([10000, 54])
torch.Size([10000, 54])
torch.Size([10000, 54])
torch.Size([10000, 54])
torch.Size([10000, 54])
torch.Size([10000, 54])
torch.Size([10000, 54])
torch.Size([10000, 54])
torch.Size([10000, 54])
torch.Size([10000, 54])
torch.Size([1000

**Prueba del modelo**

El modelo también será probado en el dataset convertido a tensores, al igual que los datos de prueba.

Podemos comprobar los resultados obtenidos del modelo

In [17]:
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    predicciones = []
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            x = x.reshape(x.shape[0], -1)

            scores = model(x)
            _, predictions = scores.max(1)
            predicciones.append(predictions)

            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    model.train()
    return num_correct/num_samples, predicciones

p_train, pred_train  = check_accuracy(train_loader, model)
p_test, pred_test  = check_accuracy(test_loader, model)

print(f"Accuracy on training set: {p_train*100:.2f}")
print(f"Accuracy on test set: {p_test*100:.2f}")

Accuracy on training set: 85.72
Accuracy on test set: 60.27
